<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/After_asked_reviews_one_2022/LSHADE/LSHADE_LPSR_codeINSIDE_F15_180_17cec2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
############################### DADOS DE ENTRADA 
 
# this code only run from F22 to F25 (you just need to change this line the pararameters and type =cool= 
#          to go to line to change IFrank, de best or your combination
RUNS=1 #NÚMERO DE RODADAS
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
ITE=1000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100
mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
# fator = 1.0/3.0; 
fator = 1.0 # just LSHADE...
mut2=0.5;crossp2=0.5;mut3=0.75;crossp3=0.25;mut4=0.25;crossp4=0.25;
mut_best  =0.5 ; crossp_best =0.5
maxNP = 180
minNP = 17
maxFES =300000
FIX = 100 # Number of adjusts random walk
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]


In [2]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz
import Go2Ann
import randomWALK
import annDE_FES_new
import classicDE_FES_new
import stepsWALK_new
#import LShade_new
from pandas import read_csv

fatal: destination path 'optim_raianars_Insight' already exists and is not an empty directory.
/content/optim_raianars_Insight


In [3]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

fatal: destination path 'Raianars_paper_One' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One


In [4]:
#import Go2Ann
#import de_soma_Insight
#import de_bounds_matriz
#import Function
import Go2MinMax
from AvaliaX import AvaliaX
import numpy as np
import scipy.stats
import random
from GeraIndices import GeraIndices
import numpy as np
import time
import matplotlib.pyplot as plt
from random import randint
from scipy import optimize
import pandas as pd
from sklearn.metrics import confusion_matrix

In [5]:
#==cool==

def LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fitness,X,Xarq,FES):

  SF,SCR,MF,MCR,p, terminal, Narquive,H = setTUNE
  # return X,BEST,FOBEST,XY,BEST_XY,FES
  #H = 6
  #kH = 0 # kH is used to work with H  
  #terminal = 0.1
  #p=0.1
  #Narquive = popsize
  #Xarq = [] # just to initialize
  #mutant= np.copy(X[0,:]) # just to initialize mutants Uij
  #fmutant = np.copy(fitness) # just to initialize fobj(Uij)
  fmutant = []
  Xmutant = []
  fx=[]; fu=[] # to select the wij Lehmer Mean.
  # setTUNE = [SF,SCR,MF,MCR,FES,p] 
  dim = len(X[0,:])
  best_number = int(p*popsize)

  
  
  termination_not_meet = True
  justGoOUT=1
  #while(termination_not_meet):
  if(justGoOUT==1):
    SCR =[];SF=[];
    for i in range(popsize):
      ri = random.randint(0,H-1) # line 7
      miF = MF[ri]
      miCR = MCR[ri]
      if(MCR[ri] == terminal):
        miCR = 0
      else:
        crossp = np.random.normal(miCR , 0.1)
      while (True):
        mut = scipy.stats.cauchy.rvs(loc=miF, scale=0.1)
        if (mut > 0):
          break
      # if(i==popsize-1): print(mut,crossp,'=====')  
      ind = fitness.argsort()[range(best_number)] # find index of best p*popsize
      best_idx = random.choice(ind) # index of the best in p*popsize (random best)
      pbest = X[best_idx] # random best 
      idxs = [idx for idx in range(popsize) if idx != i]
      a  = X[np.random.choice(idxs, 1, replace = False)]
      if(len(Xarq)>0):
        Xii = np.asarray(Xarq)
        Xnew = np.append(X,Xii,axis=0)
      else:
        Xnew=np.copy(X)
      #if(i==popsize-1): print(fbest,mut,crossp,'=====') 
      #idxs2 = [idx for idx in range(popsize+len(Xarq[:,0])) if idx != idxs]
      idxs2 = [idx for idx in range(popsize) if idx != idxs]
      b  = Xnew[np.random.choice(idxs2, 1, replace = False)]

      mutant = X[i,:]+mut*(pbest-X[i,:]) + mut * (a - b)
    
      '''
      for k in range(dim):
        if(mutant[k]>MAX[k]):
          mutant[k]=MAX[k]
        if(mutant[k]<MIN[k]):
          mutant[k]=MIN[k]
      '''  
        
      cross_points = np.random.rand(dim) < crossp
      
      #print('===== 2 =====',cross_points)
      
      if not np.any(cross_points):
        cross_points[np.random.randint(0, dim)] = True

      trial = np.where(cross_points, mutant, X[i,:])
      #mutant[i,:] = trial 
      mutant  = trial[0] 
      Xmutant.append(mutant)
      '''
      for k in range(dim):
        if(mutant[i,k]>MAX[k]):
          mutant[i,k]=MAX[k]
        if(mutant[i,k]<MIN[k]):
          mutant[i,k]=MIN[k]
      '''  
      #mutant[i,:] = trial  
      #print('===',i,mutant)
      #print('mutant ======')
      FES = FES + 1
      #xx = np.asarray(mutant)
      #fmut = fobj(xx[0])
      fmut =fobj(mutant)
      fmutant.append(fmut)
      #print('====')
      #print(fmutant)
      #print('====')
      #fmutant[i] = fobj(mutant)

      #print('===============')
      #
      #print('***********')
      #print(fmutant[0])
      #print('-----------')
      #print(fmut)
      #print(mutant)
      #print(fmutant)
      #print('--------------')
    for i in range(popsize):
      if( fmutant[i]  <= fitness[i]):
        Xold = X[i,:] 
        X[i,:] = Xmutant[i]
        fitness[i]=fmutant[i]
        if(fbest >= fmutant[i]):
          best = Xmutant[i]
          fbest = fmutant[i]
      else:
        X[i,:] = X[i,:] # it is not necessary this line but just to remeber it
      if( fmutant[i]  < fitness[i]):
        if(len(Xarq[:,0])< Narquive):
          Xarq=np.append(Xarq,Xold,axis=0)
          #Xarq = Xarq
        else:
          ri = random.randint(0, Narquive-1) # deleting individuals if necessary line 19
          Xarq[ri,:] = Xold
        #Xarq[i,:] = Xold  # need better discussion
        SCR.append(crossp)
        SF.append(mut)
        fx.append(fitness[i])
        fu.append(fmutant[i])
    # memory update algorithm 1:
    if (len(SCR)>0):
      deltaF = np.abs(np.asarray(fu)-np.asarray(fx))
      w = deltaF/sum(deltaF)
      if(MCR[kH] == terminal or max(SRC)==0):
        MCR[kH] = terminal
      else:
        y = np.asarray(SCR)
        MCR[kH] = np.dot(w,y**2)/np.dot(w,y)
      y = np.asarray(SF)
      MF[kH] = np.dot(w,y**2)/np.dot(w,y)
      kH = kH + 1
      if(kH > H): kH = 0
    else:
      MCR = MCR
      MF = MF
  termination_not_meet = False  
  
  setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]

  y=fitness

  BEST=best
  FOBEST=fbest
  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  X=XYsorted[:,0:popsize]
  XY=XYsorted
  BEST_XY =np.append(BEST,FOBEST)
  

  
  return fitness,X,BEST,FOBEST,XY,BEST_XY,FES,Xarq,setTUNE


In [6]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz

fatal: destination path 'optim_raianars_Insight' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight


In [7]:
#!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
#%cd Raianars_paper_One

!git clone https://github.com/ucfilho/Raianars_Routines #clonar do Github
%cd Raianars_Routines

fatal: destination path 'Raianars_Routines' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines


In [8]:
!git clone https://github.com/ucfilho/functions_cec2014_data
%cd functions_cec2014_data
import F1_to_F27_cec2014_zero_to_hero
import cec2014_names
import cec2014_shift_rotate

fatal: destination path 'functions_cec2014_data' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data


In [9]:
function_select = 'F15'
Function = F1_to_F27_cec2014_zero_to_hero.F15
Shift_Rotate= cec2014_shift_rotate.F15
problem_size = 30 # possibles 2, 10, 20, 30, 50, 100

In [10]:
dim = problem_size 

In [11]:
f_matrix = "M_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
shift_data_file="shift_data_"+function_select[1:]+".txt"
F_bias = cec2014_names.BiasValue()[function_select]
type_fun = cec2014_names.TypeFunctions()[function_select]
Function_CEC2014 =function_select + "_" + cec2014_names.NamesFunctions()[function_select]+"_D_"+str(problem_size)

In [12]:
!pip install --upgrade --no-cache-dir gdown

In [13]:
# !gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
# https://drive.google.com/file/d/1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz/view?usp=sharing
!gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
To: /content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/cec2014.zip
100% 36.1M/36.1M [00:00<00:00, 193MB/s]


In [14]:
import zipfile
file_name = zipfile.ZipFile('cec2014.zip', 'r')
file_name.extractall()

In [15]:
def load_shift_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values.reshape((-1))

def load_matrix_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values

In [16]:
shift_data_a = load_shift_data__(shift_data_file)[:problem_size]
matrix_a = load_matrix_data__(f_matrix)
if (type_fun == 'hybrid'):
  f_shuffle = "shuffle_data_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
  shuffle = (load_shift_data__(f_shuffle)[:problem_size] - np.ones(problem_size)).astype(int)
else:
  shuffle = matrix_a

In [17]:
SUPPORT_DIMENSION = [2, 10, 20, 30, 50, 100]
SUPPORT_DIMENSION_2 = [10, 20, 30, 50, 100]

#def Fun(solution,shift_data=shift_data_a, matrix=matrix_a, shuffle):#, bias=F_bias):
def CEC2014(solution, shift_data ,  matrix, shuffle):#, bias=F_bias):
  if(type_fun == 'shift_rotate'):
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = Function(z) + F_bias
  elif(type_fun =='hybrid'): # F17 to F22
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = z + F_bias
 
  return result

In [18]:
 shift_data = shift_data_a
 matrix = matrix_a
 shuffle = shuffle
 def Hybrid(solution):
   resp = CEC2014(solution, shift_data = shift_data_a, matrix = matrix_a, shuffle = shuffle)
   return resp

In [19]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

fatal: destination path 'Raianars_paper_One' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/Raianars_paper_One


In [20]:
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)

bounds=[]
for k in range(len(MAX)):
  bounds.append((MIN[k],MAX[k]))

In [21]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

fatal: destination path 'ANN_read_matrix' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/Raianars_paper_One/ANN_read_matrix


In [22]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')

In [23]:
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')

In [24]:
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)

In [25]:
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)

In [26]:
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')

In [27]:
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

In [28]:
##################################### 

TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

#dimensions=len(MAX)
dimensions=dim
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO


PAR=dim
Fun = Function
fobj=Fun

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
NPAR1=int(fator*NPAR);
NPAR2=int((NPAR-NPAR1)/3);
NPAR3=int((NPAR-NPAR1)/3);
NPAR4=int((NPAR-NPAR1)/3);
NPAR1 = NPAR-3*NPAR2
print('frac=',fator);
print('Npar1=',NPAR1);
print('Npar2=',NPAR2);
print('Npar3=',NPAR3);
print('Npar4=',NPAR4);
print('Npar=',NPAR)

cols=dim

X1=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X1[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
X2=np.zeros((NPAR2,cols)) # X2 is population DE (default DE)
for i in range(NPAR2):
  for j in range(cols):
    r=np.random.random()
    X2[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

X3=np.zeros((NPAR3,cols)) # X1 is Frannk population
for i in range(NPAR3):
  for j in range(cols):
    r=np.random.random()
    X3[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
X4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)
for i in range(NPAR4):
  for j in range(cols):
    r=np.random.random()
    X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
 ############################################################
  #bounds=[(5*dim, 15*dim)] * PONTOS # ITE
  #bounds=[(3*dim, 12*dim)] * PONTOS # ITE
  #bounds=[(4*dim, 16*dim)] * PONTOS
  
  # nao pode ser assim pq o bounds vale p o DE nao dos parametros
  #p=int(TOTAL/its/ITE) 
  #bounds=[(p, p)] * PONTOS


frac= 1.0
Npar1= 100
Npar2= 0
Npar3= 0
Npar4= 0
Npar= 100


In [29]:
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for iii in range(RUNS):
    FES = 0 ; gen = 0 
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)

    NPAR = maxNP
    # fator = 1.0
    NPAR1=int(fator*NPAR);
    '''
    NPAR2=int((NPAR-NPAR1)/3);
    NPAR3=int((NPAR-NPAR1)/3);
    NPAR4=int((NPAR-NPAR1)/3);
    NPAR1 = NPAR-3*NPAR2
    '''

    X1=np.zeros((NPAR1,cols)) # X1 is Frannk population
    for i in range(NPAR1):
      for j in range(cols):
        r=np.random.random()
        X1[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    '''    
    X2=np.zeros((NPAR2,cols)) # X2 is population DE (default DE)
    for i in range(NPAR2):
      for j in range(cols):
        r=np.random.random()
        X2[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

    X3=np.zeros((NPAR3,cols)) # X2 is population DE (default DE)
    for i in range(NPAR3):
      for j in range(cols):
        r=np.random.random()
        X3[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

    X4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)
    for i in range(NPAR4):
      for j in range(cols):
        r=np.random.random()
        X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    '''
    fit1 = np.asarray([fobj(ind) for ind in X1])
    best_idx = np.argmin(fit1)
    best = X1[best_idx]
    fbest = fit1[best_idx] 

    '''
    fit2 = np.asarray([fobj(ind) for ind in X2])
    fit3 = np.asarray([fobj(ind) for ind in X3])
    fit4 = np.asarray([fobj(ind) for ind in X4])
    FES = FES + NPAR1 + NPAR2 + NPAR3+ NPAR4
    '''
    FES = FES + NPAR1
    fit_best =[]
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    #Fitness = np.asarray([Fun(ind, shift_data , matrix, shuffle) for ind in X1])
    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X1,fit1 )
    nrow,ncol=X1.shape
    FES = FES + nrow

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo

    Fuma = Fun  # Fuma era Fun
  
    MAT_INDo=GeraIndices(X1,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fuma)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo

    # Just to intialize variables:
    FESr = 0
    FOBEST1 = 1E99;FOBEST2 = 1E99;FOBEST3 = 1E99;FOBEST4 = 1E99
    #BEST1,BEST2,BEST3,BEST4  =  np.copy(X1[0,:]),np.copy(X2[0,:]),np.copy(X3[0,:]), np.copy(X4[0,:]) 
    BEST1  =  np.copy(X1[0,:])
    popsize = NPAR
    FUNCTION = []
    for i in range(H):
      MCR.append(0.5);MF.append(0.5) # line 3
    Narquive = popsize
    setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]
    while(FESr == 0 ):

      #print('FES',FES)

      gen = gen +1
      ##=cool=#####################################################################################
      '''
      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2),np.copy(X2),BEST2,FOBEST2,FES
      fit2,X2,BEST2,FOBEST2,XY2,BEST_XY2,FES=classicDE_FES_new.de(bounds, mut2, crossp2,NPAR2, its,fobj,fit2,X2,FES) # de_bounds_matriz
      if(FES > maxFES):
        FESr = FES 
        fit2,X2,BEST2,FOBEST2,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
      
      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit3),np.copy(X3),BEST3,FOBEST3,FES
      fit3,X3,BEST3,FOBEST3,XY3,BEST_XY3,FES=classicDE_FES_new.de(bounds, mut3, crossp3,NPAR3, its,fobj,fit3,X3,FES) #de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit3,X3,BEST3,FOBEST3,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit4),np.copy(X4),BEST4,FOBEST4,FES
      fit4,X4,BEST4,FOBEST4,XY4,BEST_XY4,FES=classicDE_FES_new.de(bounds, mut4, crossp4,NPAR4, its,fobj,fit4,X4,FES) # de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit4,X4,BEST4,FOBEST4,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
      '''
      
      

      

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit1),np.copy(X1),BEST1,FOBEST1,FES
      Xarqback = Xarq
      #fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES = annDE_FES_new.de(MAX,MIN, mut, crossp, NPAR1, its,fobj,fit1,X1,FES)
      fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
      # fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE=LShade_new.LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
      if(FES > maxFES):
        FESr = FES 
        fit1,X1,BEST1,FOBEST1,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
        Xarq = np.copy(Xarqback)

      calc = (minNP-maxNP)/maxFES*FES+maxNP # Linear Population Size Reduction (LPSR)
      calc = int(np.round(calc)) # Linear Population Size Reduction (LPSR) 
      gen = gen + 1
      #fator = 1.0
      NPAR = calc
      NPAR1=int(fator*NPAR);
      NPAR2=int((NPAR-NPAR1)/3);
      NPAR3=int((NPAR-NPAR1)/3);
      NPAR4=int((NPAR-NPAR1)/3);
      NPAR1 = NPAR-3*NPAR2
      '''
      if(NPAR2 > len(X2[:,0])):
        NPAR2 = len(X2[:,0])
      '''
      if(NPAR1 > len(X1[:,0])):
        NPAR1 = len(X1[:,0])

      
      X1 = X1[:NPAR1,:];
      if(len(Xarq)>0) : Xarq = Xarq[:NPAR1,:]
      # X2 = X2[:NPAR2,:]; X3 = X3[:NPAR3,:]; X4 = X4[:NPAR4,:];
      fit1 = fit1[:NPAR1];# fit2 = fit2[:NPAR2]; fit3 = fit3[:NPAR3]; fit4 = fit4[:NPAR4];
      popsize = NPAR1
      Narquive = popsize
      NP = popsize
      '''
      X1[0,:] = BEST2;X1[1,:] = BEST3; X1[2,:] = BEST4;X1[3,:] = BEST1;
      X2[0,:] = BEST1;X2[1,:] = BEST3;X2[2,:] = BEST4;X2[3,:] = BEST2;
      X3[0,:] = BEST1;X3[1,:] = BEST2;X3[2,:] = BEST4;X3[3,:] = BEST3;
      X4[0,:] = BEST1;X4[1,:] = BEST2;X4[2,:] = BEST3;X4[3,:] = BEST4;
      fit1[0] = FOBEST1; fit2[1] = FOBEST2;fit3[0] = FOBEST3; fit4[1] = FOBEST4;
      '''
    '''
    for i in range(FIX):
      
        Fitback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2), np.copy(X2),BEST2,FOBEST2,FES
        
        if ( FESr == 0):
          Fitback,Xback,BESTback,FOBESTback,FESback= np.copy(X2),BEST2,FOBEST2,FES
          fit2,X2,BEST2,FOBEST2,XY2,BEST_XY2,FESr=classicDE_FES_new.de(bounds,mut_best, crossp_best,NPAR2, its,fobj,fit2,X2,FES) # de_bounds_matriz
        if ( FESr > 0):
          fit2, X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback


        if(FES > maxFES): 
          fit2, X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback
        
        
        fit2,X2,BEST2,FOBEST2,XY,BEST_XY, FES= stepsWALK_new.fixWALK(Fun,BEST2,FOBEST2,NPAR2,tunePAR,MAX,MIN,fit2,X2,FIX)
        # print('FES',FES)

      ############################################################################################
      ############################################################################################
    
    if(FES > maxFES):
      fit2,X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback

    FUNCTION.append(FOBEST2)
    '''
    #setTUNE= [SF,SCR,MF,MCR,p, terminal, Narquive]
    setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]
    print('run=',iii,'gen=',gen,'time=',(time.time()-start_time)/60, 'eval FES=', FES,'FO=',FOBEST1)  
    BEST_AGENTS.append(BEST_XY)

run= 0 gen= 8682 time= 4.4387567162513735 eval FES= 299994 FO= 1.3273034189961124e+16


In [42]:
FES,setTUNE,SCR,SF

(299994,
 [[],
  [],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  0.1,
  0.1,
  17,
  6],
 [],
 [])

In [33]:
#len(X2[1,:])
print(cols)

30


In [34]:
BEST_AGENTS=np.array(BEST_AGENTS)

In [35]:
NOMES=['DIVERSIDADE (DI)',  'DI RELATIVO','FRAC Its',
           'VELOC FOBJ','VELOC X','NP','Fo','CRo','fmin','fmax',
           'AD_fmin','AD_fmax','DELTA FOBJ','DELTA VX','r_fitness',
           'F','CR']
# BEST_INDEX= BEST_AGENTS[BEST_AGENTS[:,-1].argmin()]  
BEST_INDEX = np.argmin(BEST_AGENTS[:,-1])
#df=pd.DataFrame(MAT_stat[BEST_INDEX], columns=NOMES)



In [36]:
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()    

In [37]:
nome='SUB'+'_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('MCS-FRANNK','Fator F/DE= %.2f' %(fator),'NPAR=',NPAR,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1],'STD __:',STD)
print('MIN __:',MINFO, 'MAX__: ',MAXFO)
print(MEAN[-1])
print(STD)
print(MINFO)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)
# print('melhor=',BEST_AGENTS[BEST_INDEX])

MCS-FRANNK Fator F/DE= 1.00 NPAR= 17 Iterações= 1000 Dim= 30 Min/Max= -100 / 100 _ F15
SUB_Fo-0.5_CRo-0.5_D-30_Interv-1000 

MÉDIA:  1.3273034189961124e+16 STD __: 0.0
MIN __: 1.3273034189961124e+16 MAX__:  1.3273034189961124e+16
1.3273034189961124e+16
0.0
1.3273034189961124e+16
Tempo Médio= 266.447 segundos ---
1 Rodadas

BEST:  [ 4.72347355e+01  6.32164073e+01  1.47342884e-01 -2.42634231e+00
 -1.34094338e+01  1.03217420e+01  4.06834503e+01  5.49346743e+01
  2.35577064e+01  7.97298694e+01  2.57859219e+01 -2.76027114e+01
  1.67803642e+01 -6.15070339e+01  2.72404834e+01 -6.91516136e+01
 -3.10548241e+01  1.33452508e+01 -2.28296864e+01  7.06649153e+01
  6.09703334e+01  4.51794512e+00  7.52468393e+01  6.73217963e+01
  1.32848170e+01 -6.57372881e+01 -5.19702084e+01  2.40982506e+01
  4.12956086e+00 -3.92636007e+01  1.32730342e+16]


In [38]:
MF,MCR,SF,SCR

([0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [], [])

In [40]:
FES,Xarq

(299994, array([], dtype=float64))